In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import sys
#print(sys.path)
import time
import random
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json
from os import listdir
import glob
from scipy import spatial
#import spacy

df=pd.read_csv("Documents/data/job_sample.csv",encoding='cp850')
df1=pd.read_csv("Documents/data/survey_results_public.csv",encoding='cp850')
#print(df.head())
l=list(df["company"])
#print(type(l))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
companies=set(l)
print(len(companies))
#print(companies)
companies=list(companies)
len(companies)
#print(companies[1000])

8156


8156

In [3]:
cbf = pd.DataFrame(df1['Respondent'])
import time
start=time.time()
for i in range(98855):
    r=random.randint(1,4291)
    cbf.loc[i,"company"]=l[r]
end=time.time()
print(end-start)
print(cbf.head())

537.2178292274475
   Respondent                                    company
0           1                     Magna Infotech Pvt Ltd
1           3            InstaCar Technologies Pvt. Ltd.
2           4                        Flatworld Solutions
3           5  Lordi Systems Staffing Solutions Pvt. Ltd
4           7                              RIGHTRESOURCE


In [4]:
cbf.to_csv("Documents/data/colabdata.csv")

In [4]:
#merge all the dataset having skills from user profile.
dflang=pd.read_csv("Documents/data/LanguageWorkedWith.csv")
dfdata=pd.read_csv("Documents/data/DatabaseWorkedWith.csv")
dfplat=pd.read_csv("Documents/data/PlatformWorkedWith.csv")
dfframe=pd.read_csv("Documents/data/FrameworkWorkedWith.csv")
dfdev=pd.read_csv("Documents/data/DevType.csv")
dfmerge=pd.merge(dflang,dfdata,on="Respondent")
dfmerge=pd.merge(dfmerge,dfplat,on="Respondent")
dfmerge=pd.merge(dfmerge,dfframe,on="Respondent")
dfmerge=pd.merge(dfmerge,dfdev,on="Respondent")
#dfmerge.to_csv("Documents/data/userskills.csv")

In [5]:
import numpy as np
from numpy import array
dfuser=pd.read_csv("Documents/data/userskills.csv")
#remove duplicate columns during merge
dfuser = dfuser.loc[:,~dfuser.columns.duplicated()]
dfuser=dfuser.fillna(0)
dfuser.shape
dfuser.head()
#remove rows with less than 5 skills
dfuser=dfuser.dropna(thresh=5)
dfuser.shape
m=(np.asscalar(np.int32(max(dfuser["Respondent"]))))
print(m)
print(type(m))
#Build  a dictionary of respondent id's as keys  and thier skills as values
temp=[0]*m
vector=np.array(temp)
count=1
start=time.time()
d=dict()
for row in dfuser.iterrows():
        index,data=row
        l=list()
        #l=[data.values[0],list(data.values[1:])]
        s=np.asscalar(np.int32(data.values[0]))
        d[s]=np.array(list(data.values[1:]))
        #print(type(data))
end=time.time()
print(d[1])
print(end-start)

101592
<class 'int'>
[1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1.]
9.467660427093506


In [6]:
#Build the user user similarity matrix based on thier skill vectors
#initialise a user user matrix uptill respondent number 5000
sim=list()
s=time.time()
for i in range(5000):
        l=list()
        l=[0]*5000
        sim.append(l)
e=time.time()
print(e-s)

0.47673487663269043


In [7]:
count1=1
count2=1
t=time.time()
for key, value in d.items():
        if(key<5000):
            #print(key)
            b=(np.linalg.norm(value))
            for key2,value2 in d.items():
                if(key2<5000):
                    #print(key2)
                    a=np.dot(d[key],d[key2])
                    ans=a/(np.linalg.norm(value2)*b)
                    sim[key][key2]=ans
                    #count2+=1
            #count1+=1
e=time.time()
print("total time for building similarity matrix ")
print(e-t)

total time for building similarity matrix 
357.024533033371


In [8]:
#print(e-s)
print(sim[1][:10])
# the similarity of user 1 with others 
print((max(sim[1][2:])))
print(sim[1].index(max(sim[1][2:])))
#user 1 is most similar to user 4653 with similarity 0.71 in skills

[0.4622501635210243, 1.0000000000000002, 0.7317274774221446, 0.7190924955066748, 0.6955416066888231, 0.7432913296668157, 0.7147924077695138, 0.6821910402406466, 0.7437443194099495, 0.7088711145405624]
0.7437443194099495
8


In [9]:
#Lets check with the random jobs given to the users .
dfjob=pd.read_csv("Documents/data/colabdata.csv")
#Recommend user 3 a job based on another user who has almost the same skill as him.
print("Respondent 3 is working in ")
print(dfjob.loc[dfjob.Respondent==3]["company"].values)

m1=max(sim[3][:3])
m2=max(sim[3][4:])
ma=max(m1,m2)
suser=sim[3].index(ma)
#print(suser) #user ___ is very similar to user 3 and hence we can recommend user 3265 job to user 3.
print("Respondent ",suser,"is working in")
print(dfjob.loc[dfjob.Respondent==suser]["company"].values)

Respondent 3 is working in 
['My Connecting']
Respondent  7 is working in
['Ansal Properties & Infrastructure Ltd.']


In [10]:
#For buiding  collaborative filtering based on The Content based recommendations
dfcont=pd.read_csv("Documents/data/recommendations.csv")
sim=list()
s=time.time()
for i in range(200):
        l=list()
        l=[0]*200
        sim.append(l)
e=time.time()
print(e-s)
t=time.time()
print("Building the similarity matrix....\n")
for key, value in d.items():
        if(key<200):
            #print(key)
            b=(np.linalg.norm(value))
            for key2,value2 in d.items():
                if(key2<200):
                    #print(key2)
                    a=np.dot(d[key],d[key2])
                    ans=a/(np.linalg.norm(value2)*b)
                    sim[key][key2]=ans
                    #count2+=1
            #count1+=1
e=time.time()
print(e-t)

0.002955198287963867
Building the similarity matrix....

4.413245677947998


In [13]:
print("Respondent 3 was recommended job titles from content based filtering ")
print(dfcont.loc[dfcont.Respondent==3]["jobtitle"].values)
print("\n")
m1=max(sim[3][:3])
m2=max(sim[3][50:])
ma=max(m1,m2)
print("Similarity Accuracy: ",ma)
suser=sim[3].index(ma)
print(suser) #user 3265 is very similar to user 3 and hence we can recommend user 3265 job to user 3.
print("Respondent ",suser,"was most similiar to respondent 3\n")
print("Based on respondent ",suser," the jobs recommended to 3 are ")
#print(dfcont.loc[dfcont.Respondent==suser]["company"].values)
print("The recommended job titles are ")
jobs=dfcont.loc[dfcont.Respondent==suser]["jobtitle"].values+dfcont.loc[dfcont.Respondent==suser]["company"].values
jobs

Respondent 3 was recommended job titles from content based filtering 
['Python Developer' 'Full-Stack Python Developer'
 'Full Stack Javascript Engineer' 'Full Stack Developer'
 'System Administrator' 'System Administrator' 'Application Administrator'
 'Application Administrator' 'System Administrator' 'Full Stack Engineer']


Similarity Accuracy:  0.9343298097444914
51
Respondent  51 was most similiar to respondent 3

Based on respondent  51  the jobs recommended to 3 are 
The recommended job titles are 


array(['Software Engineer MEAN, Angular.js, Node.js, MySQLRiccione Resources, Inc.',
       'Sr. Software Engineer, The Huffington PostEdgeCast Networks, Inc.',
       'Senior Front End Web Developer - Full Time at VisaOmega Solutions Inc',
       'Hybrid Mobile Developer - HTML5, CSS3, AngularJSCyberCoders',
       'Full Time: Senior .Net Developer in Salt Lake City, UTEndure Technology Solutions, Inc.',
       'Senior Full Stack EngineerLithium Technologies, Inc.',
       'Front End Software EngineerThorndale Partners LLC',
       'Front End Software EngineerThorndale Partners LLC',
       'Front End Web DeveloperIncendia Partners',
       'Full Stack Javascript EngineerDST Systems, Inc'], dtype=object)